In [ ]:
import sys
from pathlib import Path
import functools as ft
import warnings

import pandas as pd
import numpy as np
import geopandas as gpd
import gtfs_kit as gk
import shapely
import shapely.geometry as sg
import folium as fl
import geo_kit as geo  # Only works for MRCagney staff

sys.path.append('../')

import make_gtfs as mg


DATA_DIR = Path('../data')

%load_ext autoreload
%autoreload 2
warnings.filterwarnings(action='ignore')

In [ ]:
path = DATA_DIR / 'auckland'
pfeed = mg.read_protofeed(path)
display(pfeed)



In [ ]:
#speed_zones = gpd.read_file(DATA_DIR / "auckland/speed_zones.geojson")
speed_zones = pfeed.speed_zones
# speed_zones["route_type"] = 3
# speed_zones["zone_id"] = [str(i) for i in range(speed_zones.shape[0])]
display(speed_zones)



In [ ]:
m = geo.plot(
    pfeed.speed_zones.loc[lambda x: x.route_type == 3], 
    color_by="speed_zone_id", 
    add_layer_control=False,
)
m

In [ ]:
shapes = mg.build_shapes(pfeed)
speed_zones = pfeed.speed_zones.loc[lambda x: x.route_type == 3]

shape_point_speeds = mg.compute_shape_point_speeds(shapes, speed_zones, 3)
display(shape_point_speeds.head(30))


In [ ]:
g = (
    pd.concat([gk.geometrize_shapes_0(shapes), pfeed.speed_zones.loc[lambda x: x.route_type == 3]])
    .assign(uid=lambda x: x.speed_zone_id.fillna(x.shape_id))
)          
style_fn = lambda feature: {
    "weight": 2,
    "fillOpacity": 0.3,
    "opacity": 1,
    "color": feature["properties"]["_color"],
    "fillColor": feature["properties"]["_color"],
}
geo.plot(g, group_by="uid", color_by="uid", style_function=style_fn)


In [ ]:
stops = mg.build_stops(pfeed)
stops_g = gk.geometrize_stops_0(stops, use_utm=True)
shapes_gi = gk.geometrize_shapes_0(shapes, use_utm=True).set_index("shape_id")
trip_id = "bingo"
shape_id = shapes_gi.index[0]
linestring = shapes_gi.loc[shape_id].geometry
stops_g_nearby = mg.get_stops_nearby(stops_g, linestring, "left")
route_type = 3
sz = gpd.GeoDataFrame(
    [{"speed": np.inf, "route_type": 3}], 
    geometry=[sg.box(*linestring.bounds).buffer(10)], 
    crs=stops_g.crs,
)
shape_point_speeds = mg.compute_shape_point_speeds(shapes, sz, 3)

st = mg.build_stop_times_for_trip(
    "bingo", 
    stops_g_nearby, 
    shape_id, 
    linestring, 
    sz,
    route_type,
    shape_point_speeds, 
    default_speed=2, 
    start_time=0,
)
display(st)

speed = (
    3.6 * (st.shape_dist_traveled.iat[-1] - st.shape_dist_traveled.iat[0])
    / st.departure_time.iat[-1]
)
speed

In [ ]:
agency = mg.build_agency(pfeed)
calendar, service_by_window = mg.build_calendar_etc(pfeed)
routes = mg.build_routes(pfeed)
shapes = mg.build_shapes(pfeed)
stops = mg.build_stops(pfeed, shapes)
trips = mg.build_trips(pfeed, routes, service_by_window)

%time st = mg.build_stop_times(pfeed, routes, shapes, stops, trips)

st

In [ ]:
feed = mg.build_feed(pfeed)
ts = feed.compute_trip_stats()
ts.speed.value_counts()

In [ ]:
# Map some trips
prefix = 't-rB-weekday_peak_1-07:00:00'
t = (
    feed.trips
    .loc[lambda x: x.trip_id.str.startswith(prefix)]
)
trip_id = t.trip_id.iat[0]



In [ ]:
# Map some trips
prefix = 't-rB-weekday_peak_1-07:00:00'
t = (
    feed.trips
    .loc[lambda x: x.trip_id.str.startswith(prefix)]
)
tids = [t.trip_id.iat[0]]
feed.map_trips(tids, include_arrows=True, include_stops=True)
    